In [8]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from itertools import zip_longest
import os
import glob
import ast

In [2]:
# data = json.load(open("2025-06-15_11-38-56-781937_measure_weight_FORCE_MEASUREMENT_CASE1.json", encoding="utf-8"))
data = json.load(open("2025-06-15_11-39-04-387039_measure_weight_FORCE_MEASUREMENT_CASE.json", encoding="utf-8"))

df = pd.json_normalize(
    data,
    record_path="raw_measurements",
    meta=["version","type","start_time","end_time","sampling_rate","weight_filter","zero_offset","expected_weight","robot_type","force_sensor_offset"],
    errors="ignore"
)

# drop useless columns
df = df.drop('robot_tcp.timestamp', axis=1) # robot time, not related to the data.
df = df.drop('version', axis=1) # not a measurement
df = df.drop('type', axis=1) # not a measurement
df = df.drop('start_time', axis=1) # real time not important
df = df.drop('end_time', axis=1) # real time not important
df = df.drop('sampling_rate', axis=1) # assumed to be always 1000
df = df.drop('robot_type', axis=1) # not a measurement
df = df.drop('weight_filter', axis=1) # not a measurement

# start the time from 0
df["timestamp"] = df["timestamp"] - df["timestamp"].iloc[0]


# split the force vector
_fv = pd.DataFrame(
    df["force_vector"].apply(
        lambda x: x if isinstance(x, (list, tuple)) else [np.nan, np.nan, np.nan]
    ).tolist(),
    columns=["force_vector.Mx", "force_vector.My", "force_vector.Fz"],
    index=df.index,
)
df = pd.concat([df, _fv], axis=1)


# split velocity vector 
_vl = pd.DataFrame(
    df["robot_tcp.velocity_linear"].apply(
        lambda x: x if isinstance(x, (list, tuple)) else [np.nan, np.nan, np.nan]
    ).tolist(),
    columns=["robot_tcp.velocity_linear.Vx", "robot_tcp.velocity_linear.Vy", "robot_tcp.velocity_linear.Vz"],
    index=df.index,
)
df = pd.concat([df, _vl], axis=1)


# split velocity vector 
_va = pd.DataFrame(
    df["robot_tcp.velocity_angular"].apply(
        lambda x: x if isinstance(x, (list, tuple)) else [np.nan, np.nan, np.nan]
    ).tolist(),
    columns=["robot_tcp.velocity_angular.Wx", "robot_tcp.velocity_angular.Wy", "robot_tcp.velocity_angular.Wz"],
    index=df.index,
)
df = pd.concat([df, _va], axis=1)


# split angle vector 
_ja = pd.DataFrame(
    df["robot_tcp.joint_angles"].apply(
        lambda x: x if isinstance(x, (list, tuple)) else [np.nan]*6
    ).tolist(),
    columns=["robot_tcp.joint_angles.base","robot_tcp.joint_angles.shoulder","robot_tcp.joint_angles.elbow",
             "robot_tcp.joint_angles.wrist_1","robot_tcp.joint_angles.wrist_2","robot_tcp.joint_angles.wrist_3"],
    index=df.index,
)
df = pd.concat([df, _ja], axis=1)


# split flange vector 
_t = pd.DataFrame(
    df["robot_tcp.flange"].apply(
        lambda x: x[0] if isinstance(x, (list, tuple)) and len(x) > 0 else [np.nan, np.nan, np.nan]
    ).tolist(),
    columns=["robot_tcp.flange.tx","robot_tcp.flange.ty","robot_tcp.flange.tz"],
    index=df.index,
)
_q = pd.DataFrame(
    df["robot_tcp.flange"].apply(
        lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else [np.nan, np.nan, np.nan, np.nan]
    ).tolist(),
    columns=["robot_tcp.flange.qw","robot_tcp.flange.qx","robot_tcp.flange.qy","robot_tcp.flange.qz"],
    index=df.index,
)
df = pd.concat([df, _t, _q], axis=1)


# split offset
_t = pd.DataFrame(
    df["robot_tcp.tcp_offset"].apply(
        lambda x: x[0] if isinstance(x, (list, tuple)) and len(x) > 0 else [np.nan, np.nan, np.nan]
    ).tolist(),
    columns=["robot_tcp.tcp_offset.tx","robot_tcp.tcp_offset.ty","robot_tcp.tcp_offset.tz"],
    index=df.index,
)
_q = pd.DataFrame(
    df["robot_tcp.tcp_offset"].apply(
        lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else [np.nan, np.nan, np.nan, np.nan]
    ).tolist(),
    columns=["robot_tcp.tcp_offset.qw","robot_tcp.tcp_offset.qx","robot_tcp.tcp_offset.qy","robot_tcp.tcp_offset.qz"],
    index=df.index,
)
df = pd.concat([df, _t, _q], axis=1)


# split sensor offset
_t = pd.DataFrame(
    df["force_sensor_offset"].apply(
        lambda x: x[0] if isinstance(x, (list, tuple)) and len(x) > 0 else [np.nan, np.nan, np.nan]
    ).tolist(),
    columns=["force_sensor_offset.tx","force_sensor_offset.ty","force_sensor_offset.tz"],
    index=df.index,
)
_q = pd.DataFrame(
    df["force_sensor_offset"].apply(
        lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else [np.nan, np.nan, np.nan, np.nan]
    ).tolist(),
    columns=["force_sensor_offset.qw","force_sensor_offset.qx","force_sensor_offset.qy","force_sensor_offset.qz"],
    index=df.index,
)
df = pd.concat([df, _t, _q], axis=1)


# drop the parent columns
_drop = [
    "force_vector",
    "robot_tcp.velocity_linear",
    "robot_tcp.velocity_angular",
    "robot_tcp.joint_angles",
    "robot_tcp.flange",
    "robot_tcp.tcp_offset",
    "eoat_params",
    "force_sensor_offset",
]
df = df.drop(columns=[c for c in _drop if c in df.columns])

In [3]:
print(df.keys())

Index(['timestamp', 'value', 'zero_offset', 'expected_weight',
       'force_vector.Mx', 'force_vector.My', 'force_vector.Fz',
       'robot_tcp.velocity_linear.Vx', 'robot_tcp.velocity_linear.Vy',
       'robot_tcp.velocity_linear.Vz', 'robot_tcp.velocity_angular.Wx',
       'robot_tcp.velocity_angular.Wy', 'robot_tcp.velocity_angular.Wz',
       'robot_tcp.joint_angles.base', 'robot_tcp.joint_angles.shoulder',
       'robot_tcp.joint_angles.elbow', 'robot_tcp.joint_angles.wrist_1',
       'robot_tcp.joint_angles.wrist_2', 'robot_tcp.joint_angles.wrist_3',
       'robot_tcp.flange.tx', 'robot_tcp.flange.ty', 'robot_tcp.flange.tz',
       'robot_tcp.flange.qw', 'robot_tcp.flange.qx', 'robot_tcp.flange.qy',
       'robot_tcp.flange.qz', 'robot_tcp.tcp_offset.tx',
       'robot_tcp.tcp_offset.ty', 'robot_tcp.tcp_offset.tz',
       'robot_tcp.tcp_offset.qw', 'robot_tcp.tcp_offset.qx',
       'robot_tcp.tcp_offset.qy', 'robot_tcp.tcp_offset.qz',
       'force_sensor_offset.tx', 'force_sens

In [4]:
df

,timestamp,value,zero_offset,expected_weight,force_vector.Mx,force_vector.My,force_vector.Fz,robot_tcp.velocity_linear.Vx,robot_tcp.velocity_linear.Vy,robot_tcp.velocity_linear.Vz,...,robot_tcp.tcp_offset.qx,robot_tcp.tcp_offset.qy,robot_tcp.tcp_offset.qz,force_sensor_offset.tx,force_sensor_offset.ty,force_sensor_offset.tz,force_sensor_offset.qw,force_sensor_offset.qx,force_sensor_offset.qy,force_sensor_offset.qz
0,0.000000,-0.054,-0.111579,0.118,0.603,0.618,9.0,0.015086,0.003652,0.888726,...,0.0,0.0,0.0,0.0,0.0,0.02,1.0,0.0,0.0,0.0
1,0.001043,-0.055,-0.111579,0.118,0.647,0.590,9.0,0.015086,0.003652,0.888726,...,0.0,0.0,0.0,0.0,0.0,0.02,1.0,0.0,0.0,0.0
2,0.002042,-0.055,-0.111579,0.118,0.683,0.560,10.0,0.015086,0.003652,0.888726,...,0.0,0.0,0.0,0.0,0.0,0.02,1.0,0.0,0.0,0.0
3,0.003042,-0.055,-0.111579,0.118,0.711,0.529,11.0,0.015086,0.003652,0.888726,...,0.0,0.0,0.0,0.0,0.0,0.02,1.0,0.0,0.0,0.0
4,0.004042,-0.055,-0.111579,0.118,0.730,0.502,11.0,0.015086,0.003652,0.888726,...,0.0,0.0,0.0,0.0,0.0,0.02,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,0.495507,0.060,-0.111579,0.118,1.102,-0.557,16.0,0.261455,0.329862,0.982952,...,0.0,0.0,0.0,0.0,0.0,0.02,1.0,0.0,0.0,0.0
489,0.496506,0.060,-0.111579,0.118,1.106,-0.540,17.0,0.261455,0.329862,0.982952,...,0.0,0.0,0.0,0.0,0.0,0.02,1.0,0.0,0.0,0.0
490,0.497508,0.061,-0.111579,0.118,1.095,-0.524,17.0,0.261455,0.329862,0.982952,...,0.0,0.0,0.0,0.0,0.0,0.02,1.0,0.0,0.0,0.0
491,0.498506,0.062,-0.111579,0.118,1.072,-0.500,18.0,0.261455,0.329862,0.982952,...,0.0,0.0,0.0,0.0,0.0,0.02,1.0,0.0,0.0,0.0


In [5]:
# 1. Separate constants and non-constants
constant_columns = [col for col in df.columns if df[col].nunique(dropna=True) == 1 or df[col].isna().all()]
non_constant_columns = [col for col in df.columns if col not in constant_columns]

# 2. Grab the constant values
constant_values = {col: df[col].dropna().iloc[0] if not df[col].isna().all() else float('nan')
                   for col in constant_columns}

# 3. Compute the mean for non-constant columns (numeric only)
means = df[non_constant_columns].mean(numeric_only=True)

# 4. Build a one-row DataFrame
row = {**constant_values, **means.to_dict()}
df_one_row = pd.DataFrame([row], columns=df.columns)

print("Non-constant columns:", non_constant_columns)
print("Constant columns:", constant_columns)
df_one_row

Non-constant columns: ['timestamp', 'value', 'force_vector.Mx', 'force_vector.My', 'force_vector.Fz', 'robot_tcp.velocity_linear.Vx', 'robot_tcp.velocity_linear.Vy', 'robot_tcp.velocity_linear.Vz', 'robot_tcp.velocity_angular.Wx', 'robot_tcp.velocity_angular.Wy', 'robot_tcp.velocity_angular.Wz', 'robot_tcp.joint_angles.base', 'robot_tcp.joint_angles.shoulder', 'robot_tcp.joint_angles.elbow', 'robot_tcp.joint_angles.wrist_1', 'robot_tcp.joint_angles.wrist_2', 'robot_tcp.joint_angles.wrist_3', 'robot_tcp.flange.tx', 'robot_tcp.flange.ty', 'robot_tcp.flange.tz', 'robot_tcp.flange.qw', 'robot_tcp.flange.qx', 'robot_tcp.flange.qy', 'robot_tcp.flange.qz']
Constant columns: ['zero_offset', 'expected_weight', 'robot_tcp.tcp_offset.tx', 'robot_tcp.tcp_offset.ty', 'robot_tcp.tcp_offset.tz', 'robot_tcp.tcp_offset.qw', 'robot_tcp.tcp_offset.qx', 'robot_tcp.tcp_offset.qy', 'robot_tcp.tcp_offset.qz', 'force_sensor_offset.tx', 'force_sensor_offset.ty', 'force_sensor_offset.tz', 'force_sensor_offset.q

,timestamp,value,zero_offset,expected_weight,force_vector.Mx,force_vector.My,force_vector.Fz,robot_tcp.velocity_linear.Vx,robot_tcp.velocity_linear.Vy,robot_tcp.velocity_linear.Vz,...,robot_tcp.tcp_offset.qx,robot_tcp.tcp_offset.qy,robot_tcp.tcp_offset.qz,force_sensor_offset.tx,force_sensor_offset.ty,force_sensor_offset.tz,force_sensor_offset.qw,force_sensor_offset.qx,force_sensor_offset.qy,force_sensor_offset.qz
0,0.249457,-0.018542,-0.111579,0.118,0.751414,0.766909,14.411765,0.22059,-0.119075,0.8839,...,0.0,0.0,0.0,0.0,0.0,0.02,1.0,0.0,0.0,0.0


In [6]:
# expected_weight is the important one.

In [7]:
base_dir = Path("All measurements, cleaned")

rows = []
failed = []

# define columns we must have for processing
required_columns = [
    "force_vector",
    "robot_tcp.velocity_linear",
    "robot_tcp.velocity_angular",
    "robot_tcp.joint_angles",
    "robot_tcp.flange",
    "robot_tcp.tcp_offset",
    "force_sensor_offset"
]

for fp in sorted(base_dir.glob("*.json")):
    try:
        data = json.load(open(fp, encoding="utf-8"))
        df = pd.json_normalize(
            data,
            record_path="raw_measurements",
            meta=[
                "version","type","start_time","end_time","sampling_rate",
                "weight_filter","zero_offset","expected_weight","robot_type",
                "force_sensor_offset"
            ],
            errors="ignore"
        )

        # check all required columns exist
        if not all(col in df.columns for col in required_columns):
            print(f"Skipping {fp.name}: missing required columns")
            continue

        # drop useless columns
        drop_cols = [
            "robot_tcp.timestamp","version","type","start_time","end_time",
            "sampling_rate","robot_type","weight_filter"
        ]
        df = df.drop(columns=[c for c in drop_cols if c in df.columns])

        # reset timestamp
        df["timestamp"] = df["timestamp"] - df["timestamp"].iloc[0]

        # --- split nested structures (no ifs, since we checked above) ---
        _fv = pd.DataFrame(df["force_vector"].tolist(),
                           columns=["force_vector.Mx","force_vector.My","force_vector.Fz"],
                           index=df.index)
        df = pd.concat([df, _fv], axis=1)

        _vl = pd.DataFrame(df["robot_tcp.velocity_linear"].tolist(),
                           columns=["robot_tcp.velocity_linear.Vx","robot_tcp.velocity_linear.Vy","robot_tcp.velocity_linear.Vz"],
                           index=df.index)
        df = pd.concat([df, _vl], axis=1)

        _va = pd.DataFrame(df["robot_tcp.velocity_angular"].tolist(),
                           columns=["robot_tcp.velocity_angular.Wx","robot_tcp.velocity_angular.Wy","robot_tcp.velocity_angular.Wz"],
                           index=df.index)
        df = pd.concat([df, _va], axis=1)

        _ja = pd.DataFrame(df["robot_tcp.joint_angles"].tolist(),
                           columns=["robot_tcp.joint_angles.base","robot_tcp.joint_angles.shoulder","robot_tcp.joint_angles.elbow",
                                    "robot_tcp.joint_angles.wrist_1","robot_tcp.joint_angles.wrist_2","robot_tcp.joint_angles.wrist_3"],
                           index=df.index)
        df = pd.concat([df, _ja], axis=1)

        _t = pd.DataFrame(df["robot_tcp.flange"].apply(lambda x: x[0]).tolist(),
                          columns=["robot_tcp.flange.tx","robot_tcp.flange.ty","robot_tcp.flange.tz"],
                          index=df.index)
        _q = pd.DataFrame(df["robot_tcp.flange"].apply(lambda x: x[1]).tolist(),
                          columns=["robot_tcp.flange.qw","robot_tcp.flange.qx","robot_tcp.flange.qy","robot_tcp.flange.qz"],
                          index=df.index)
        df = pd.concat([df, _t, _q], axis=1)

        _t = pd.DataFrame(df["robot_tcp.tcp_offset"].apply(lambda x: x[0]).tolist(),
                          columns=["robot_tcp.tcp_offset.tx","robot_tcp.tcp_offset.ty","robot_tcp.tcp_offset.tz"],
                          index=df.index)
        _q = pd.DataFrame(df["robot_tcp.tcp_offset"].apply(lambda x: x[1]).tolist(),
                          columns=["robot_tcp.tcp_offset.qw","robot_tcp.tcp_offset.qx","robot_tcp.tcp_offset.qy","robot_tcp.tcp_offset.qz"],
                          index=df.index)
        df = pd.concat([df, _t, _q], axis=1)

        _t = pd.DataFrame(df["force_sensor_offset"].apply(lambda x: x[0]).tolist(),
                          columns=["force_sensor_offset.tx","force_sensor_offset.ty","force_sensor_offset.tz"],
                          index=df.index)
        _q = pd.DataFrame(df["force_sensor_offset"].apply(lambda x: x[1]).tolist(),
                          columns=["force_sensor_offset.qw","force_sensor_offset.qx","force_sensor_offset.qy","force_sensor_offset.qz"],
                          index=df.index)
        df = pd.concat([df, _t, _q], axis=1)

        # drop parent list columns
        df = df.drop(columns=[c for c in required_columns if c in df.columns] + ["eoat_params"])

        # --- aggregate to one row ---
        constant_columns = [c for c in df.columns if df[c].nunique(dropna=True) == 1 or df[c].isna().all()]
        non_constant_columns = [c for c in df.columns if c not in constant_columns]

        constant_values = {c: df[c].dropna().iloc[0] if not df[c].isna().all() else float('nan') 
                           for c in constant_columns}
        means = df[non_constant_columns].mean(numeric_only=True)

        row = {**constant_values, **means.to_dict()}
        row["_source_file"] = fp.name
        rows.append(row)

    except Exception as e:
        failed.append((fp.name, repr(e)))

combined_df = pd.DataFrame(rows)
if "_source_file" in combined_df.columns:
    cols = ["_source_file"] + [c for c in combined_df.columns if c != "_source_file"]
    combined_df = combined_df[cols]

print(f"Processed files: {len(rows)}")
if failed:
    print("Failed files:")
    for name, err in failed:
        print(f"  - {name}: {err}")

Processed files: 0


In [8]:
combined_df.to_csv("all_measurements_summary.csv", index=False)

In [9]:
from pathlib import Path

folder_path = Path("All measurements, cleaned")  # replace with your folder path
files = list(folder_path.iterdir())
print("Number of files:", len(files))


Number of files: 16797


In [5]:
df = pd.read_csv(r"All measurements, cleaned\2025-10-01_11-02-40-091553_measure_weight_FORCE_MEASUREMENT_CASE_cleaned.csv")

df.keys()

Index(['force_vector', 'value', 'timestamp', 'robot_tcp.velocity_linear',
       'robot_tcp.velocity_angular', 'robot_tcp.timestamp', 'robot_tcp.flange',
       'robot_tcp.tcp_offset', 'robot_tcp.joint_angles', 'version', 'type',
       'start_time', 'end_time', 'sampling_rate', 'weight_filter',
       'zero_offset', 'expected_weight', 'robot_type', 'force_sensor_offset'],
      dtype='object')

In [9]:
base_dir = Path("All measurements, cleaned")

print("Scanning:", base_dir.resolve())

total_entries = 0
csv_ok = 0
has_raw = 0
passed_required = 0
processed = 0

rows, failed, skipped = [], [], []

# columns required; if any missing -> skip the entire file
required_columns = [
    "force_vector",
    "robot_tcp.velocity_linear",
    "robot_tcp.velocity_angular",
    "robot_tcp.joint_angles",
    "robot_tcp.flange",
    "robot_tcp.tcp_offset",
    "force_sensor_offset",
    "timestamp",
]

for fp in sorted(base_dir.iterdir()):
    if not fp.is_file():
        continue
    total_entries += 1

    try:   
        # read CSV (no raw_measurements / no json_normalize)
        df = pd.read_csv(fp, low_memory=False)
        csv_ok += 1

        # check required columns exist
        if not all(col in df.columns for col in required_columns):
            missing = [col for col in required_columns if col not in df.columns]
            skipped.append((fp.name, f"missing required cols: {missing}"))
            continue
        passed_required += 1

        # drop useless columns
        drop_cols = [
            "robot_tcp.timestamp","version","type","start_time","end_time",
            "sampling_rate","robot_type","weight_filter"
        ]
        df = df.drop(columns=[c for c in drop_cols if c in df.columns])

        # reset timestamp to start at zero
        df["timestamp"] = df["timestamp"] - df["timestamp"].iloc[0]

        for i in required_columns:
            df[i] = df[i].apply(
                lambda x: ast.literal_eval(x) if isinstance(x, str) else x
            )

        
        _fv = pd.DataFrame(df["force_vector"].tolist(),
                           columns=["force_vector.Mx","force_vector.My","force_vector.Fz"],
                           index=df.index)
        df = pd.concat([df, _fv], axis=1)

        _vl = pd.DataFrame(df["robot_tcp.velocity_linear"].tolist(),
                           columns=["robot_tcp.velocity_linear.Vx","robot_tcp.velocity_linear.Vy","robot_tcp.velocity_linear.Vz"],
                           index=df.index)
        df = pd.concat([df, _vl], axis=1)


        _va = pd.DataFrame(df["robot_tcp.velocity_angular"].tolist(),
                           columns=["robot_tcp.velocity_angular.Wx","robot_tcp.velocity_angular.Wy","robot_tcp.velocity_angular.Wz"],
                           index=df.index)
        df = pd.concat([df, _va], axis=1)
        
        _ja = pd.DataFrame(df["robot_tcp.joint_angles"].tolist(),
                           columns=[
                               "robot_tcp.joint_angles.base","robot_tcp.joint_angles.shoulder","robot_tcp.joint_angles.elbow",
                               "robot_tcp.joint_angles.wrist_1","robot_tcp.joint_angles.wrist_2","robot_tcp.joint_angles.wrist_3"
                           ],
                           index=df.index)
        df = pd.concat([df, _ja], axis=1)
        
        _t = pd.DataFrame(df["robot_tcp.flange"].apply(lambda x: x[0]).tolist(),
                          columns=["robot_tcp.flange.tx","robot_tcp.flange.ty","robot_tcp.flange.tz"],
                          index=df.index)
        _q = pd.DataFrame(df["robot_tcp.flange"].apply(lambda x: x[1]).tolist(),
                          columns=["robot_tcp.flange.qw","robot_tcp.flange.qx","robot_tcp.flange.qy","robot_tcp.flange.qz"],
                          index=df.index)
        df = pd.concat([df, _t, _q], axis=1)
        
        _t = pd.DataFrame(df["robot_tcp.tcp_offset"].apply(lambda x: x[0]).tolist(),
                          columns=["robot_tcp.tcp_offset.tx","robot_tcp.tcp_offset.ty","robot_tcp.tcp_offset.tz"],
                          index=df.index)
        _q = pd.DataFrame(df["robot_tcp.tcp_offset"].apply(lambda x: x[1]).tolist(),
                          columns=["robot_tcp.tcp_offset.qw","robot_tcp.tcp_offset.qx","robot_tcp.tcp_offset.qy","robot_tcp.tcp_offset.qz"],
                          index=df.index)
        df = pd.concat([df, _t, _q], axis=1)

        _t = pd.DataFrame(df["force_sensor_offset"].apply(lambda x: x[0]).tolist(),
                          columns=["force_sensor_offset.tx","force_sensor_offset.ty","force_sensor_offset.tz"],
                          index=df.index)
        _q = pd.DataFrame(df["force_sensor_offset"].apply(lambda x: x[1]).tolist(),
                          columns=["force_sensor_offset.qw","force_sensor_offset.qx","force_sensor_offset.qy","force_sensor_offset.qz"],
                          index=df.index)
        df = pd.concat([df, _t, _q], axis=1)
        
        # drop parent composites
        parent_cols = [
            "force_vector",
            "robot_tcp.velocity_linear",
            "robot_tcp.velocity_angular",
            "robot_tcp.joint_angles",
            "robot_tcp.flange",
            "robot_tcp.tcp_offset",
            "force_sensor_offset",
        ]
        df = df.drop(columns=[c for c in parent_cols if c in df.columns])

        # aggregate one row
        constant_columns = [c for c in df.columns if df[c].nunique(dropna=True) == 1 or df[c].isna().all()]
        non_constant_columns = [c for c in df.columns if c not in constant_columns]

        constant_values = {c: (df[c].dropna().iloc[0] if not df[c].isna().all() else float('nan'))
                           for c in constant_columns}
        means = df[non_constant_columns].mean(numeric_only=True)

        row = {**constant_values, **means.to_dict()}
        row["_source_file"] = fp.name
        rows.append(row)
        processed += 1

    except Exception as e:
        failed.append((fp.name, repr(e)))
        continue

combined_df = pd.DataFrame(rows)
if "_source_file" in combined_df.columns:
    combined_df = combined_df[["_source_file"] + [c for c in combined_df.columns if c != "_source_file"]]

print("\n--- DIAGNOSTICS ---")
print(f"Total entries seen: {total_entries}")
print(f"CSV parsed OK:      {csv_ok}")
print(f"With raw_measurements: {has_raw}")
print(f"Passed required cols:  {passed_required}")
print(f"Processed rows:        {processed}")
print(f"Skipped files:         {len(skipped)}")
print(f"Failed files:          {len(failed)}")
if skipped[:10]:
    print("\nExamples of skipped:")
    for name, reason in skipped[:10]:
        print(f"  - {name}: {reason}")
if failed[:5]:
    print("\nExamples of failed:")
    for name, err in failed[:5]:
        print(f"  - {name}: {err}")

# combined_df now has one row per processed file
combined_df


Scanning: C:\Users\piete\Documents\Smart Robotics PROJECT\All measurements, cleaned

--- DIAGNOSTICS ---
Total entries seen: 16797
CSV parsed OK:      16797
With raw_measurements: 0
Passed required cols:  16797
Processed rows:        16797
Skipped files:         0
Failed files:          0


,_source_file,zero_offset,expected_weight,robot_tcp.tcp_offset.tx,robot_tcp.tcp_offset.ty,robot_tcp.tcp_offset.tz,robot_tcp.tcp_offset.qw,robot_tcp.tcp_offset.qx,robot_tcp.tcp_offset.qy,robot_tcp.tcp_offset.qz,...,robot_tcp.joint_angles.wrist_1,robot_tcp.joint_angles.wrist_2,robot_tcp.joint_angles.wrist_3,robot_tcp.flange.tx,robot_tcp.flange.ty,robot_tcp.flange.tz,robot_tcp.flange.qw,robot_tcp.flange.qx,robot_tcp.flange.qy,robot_tcp.flange.qz
0,2025-06-15_11-44-34-096154_measure_weight_FORC...,-0.111534,0.105,0.0,0.0,0.4,1.0,0.0,0.0,0.0,...,-2.190106,4.691436,-1.922700,-0.532655,-0.744487,0.229433,-0.530563,0.847511,0.013933,0.004224
1,2025-06-15_11-44-43-798071_measure_weight_FORC...,-0.106988,0.318,0.0,0.0,0.4,1.0,0.0,0.0,0.0,...,-2.539432,4.707798,-3.600763,-0.293113,-0.559246,0.205482,0.988812,-0.148761,-0.009088,0.005305
2,2025-06-15_11-45-14-855432_measure_weight_FORC...,-0.113433,0.154,0.0,0.0,0.4,1.0,0.0,0.0,0.0,...,-2.405048,4.692700,-1.831010,-0.267612,-0.529407,0.234284,-0.510403,0.859409,0.017731,0.020974
3,2025-06-15_11-45-24-472373_measure_weight_FORC...,-0.115961,0.254,0.0,0.0,0.4,1.0,0.0,0.0,0.0,...,-2.305532,4.729643,-1.224130,-0.423766,-0.540051,0.275155,-0.151717,0.988389,-0.002819,0.007212
4,2025-06-15_11-48-35-226293_measure_weight_FORC...,-0.014848,0.165,0.0,0.0,0.4,1.0,0.0,0.0,0.0,...,-2.140639,4.731679,-3.316016,-0.308994,-0.667908,0.354600,0.969513,-0.244890,0.006659,-0.000032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16792,2025-10-01_11-01-20-731031_measure_weight_FORC...,-0.102833,NaN,0.0,0.0,0.4,1.0,0.0,0.0,0.0,...,-1.559613,4.719714,-3.375033,-0.125538,-0.445232,0.622676,0.977339,-0.211682,0.000005,-0.000011
16793,2025-10-01_11-02-03-980754_measure_weight_FORC...,-0.182765,NaN,0.0,0.0,0.4,1.0,0.0,0.0,0.0,...,-2.704978,4.734760,-1.451666,-0.331521,-0.380680,0.140933,-0.186595,0.982121,-0.002324,0.023749
16794,2025-10-01_11-02-12-507100_measure_weight_FORC...,-0.138998,NaN,0.0,0.0,0.4,1.0,0.0,0.0,0.0,...,-2.403008,4.688512,-3.654032,-0.205225,-0.535427,0.264792,0.997035,-0.075448,-0.009256,-0.011476
16795,2025-10-01_11-02-30-043520_measure_weight_FORC...,-0.160860,NaN,0.0,0.0,0.4,1.0,0.0,0.0,0.0,...,-2.658055,4.701482,-4.199409,-0.269701,-0.505755,0.163638,0.991255,0.131698,-0.004597,-0.005054


In [11]:
combined_df.to_csv("all_measurements_summary.csv", index=False)